In [1]:
# Code from 
import torch
import math

dtype = torch.float
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)
c = torch.randn((), dtype=dtype, requires_grad=True)
d = torch.randn((), dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

/home/jessica/miniconda3/envs/torch-test/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


99 2277.735107421875
199 1580.0631103515625
299 1097.9927978515625
399 764.551025390625
499 533.6798095703125
599 373.66845703125
699 262.6610107421875
799 185.578125
899 132.00323486328125
999 94.73424530029297
1099 68.7862319946289
1199 50.70525360107422
1299 38.09626388549805
1399 29.2965087890625
1499 23.150659561157227
1599 18.855321884155273
1699 15.851286888122559
1799 13.74896240234375
1899 12.276814460754395
1999 11.245322227478027
Result: y = 0.04898932948708534 + 0.8402983546257019 x + -0.008451475761830807 x^2 + -0.09099159389734268 x^3


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("stock-data.csv")
df.info()
df.drop("Date", inplace=True, axis = 1)
df = df.replace(",","", regex = True)
df = df.astype(float)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    232 non-null    object
 1   Open    232 non-null    object
 2   High    232 non-null    object
 3   Low     232 non-null    object
 4   Close   232 non-null    object
dtypes: object(5)
memory usage: 9.2+ KB


In [3]:
features = df[["Open", "High", "Low"]]
print(features.info())
features = features.values
target = df[["Close"]].values
# features.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    232 non-null    float64
 1   High    232 non-null    float64
 2   Low     232 non-null    float64
dtypes: float64(3)
memory usage: 5.6 KB
None


In [4]:
# mean = features.mean(axis = 0)
# std = features.std(axis=0)
# features = (features - mean)/std

features = torch.round(torch.tensor(features, dtype = torch.float64)*100/100)

#this is beacuse we want a 1D array for our answerers
target = torch.round(torch.tensor(target, dtype = torch.float64).view(-1,1)*100)/100


In [5]:
import torch.nn as nn
import torch.nn.functional as F

class stockPredictor(nn.Module):
    def __init__(self):
        super(stockPredictor, self).__init__()
        #this data has 4 inputs than we want to go to 64 i dont not understand why we chose these numbers
        self.fc1 = nn.Linear(3,16)
        self.fc2 = nn.Linear(16,63)
        self.fc3 = nn.Linear(63,32)
        self.fc5 = nn.Linear(32,1)
        self.double()
    
    #goes through each layer of the neural netwrok and returns an answer depending
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = stockPredictor()

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.15, random_state = 41)
print(y_test)

tensor([[4536.3400],
        [5234.1800],
        [4754.6300],
        [5307.0100],
        [5070.5500],
        [5071.6300],
        [5204.3400],
        [5178.5100],
        [4845.6500],
        [5243.7700],
        [4719.1900],
        [4585.5900],
        [4890.9700],
        [4411.5900],
        [4643.7000],
        [4193.8000],
        [4508.2400],
        [5199.0600],
        [5308.1500],
        [4522.7900],
        [5104.7600],
        [4997.9100],
        [4186.7700],
        [4373.6300],
        [5149.4200],
        [4247.6800],
        [4465.4800],
        [4376.3100],
        [5137.0800],
        [5203.5800],
        [4274.5100],
        [5087.0300],
        [4468.8300],
        [4505.4200],
        [5096.2700]], dtype=torch.float64)


In [7]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr = 0.001)

epochs = 500
losses = []
for epoch in range(epochs):

    y_pred = model.forward(x_train)

    outputs = model(features)
    loss = criterion(y_pred, y_train)

    losses.append(loss.detach().numpy())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch +1) %10 ==0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss}')

/home/jessica/miniconda3/envs/torch-test/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Epoch [10/500], Loss: 19647658.400308494
Epoch [20/500], Loss: 16390940.253635852
Epoch [30/500], Loss: 11285505.318657793
Epoch [40/500], Loss: 4492711.204695241
Epoch [50/500], Loss: 57291.40469835453
Epoch [60/500], Loss: 605170.870265514
Epoch [70/500], Loss: 3712.9287641215537
Epoch [80/500], Loss: 76018.54899937924
Epoch [90/500], Loss: 2679.2273216165695
Epoch [100/500], Loss: 9931.838577938415
Epoch [110/500], Loss: 425.59076679619943
Epoch [120/500], Loss: 1741.383994205673
Epoch [130/500], Loss: 409.35777031793634
Epoch [140/500], Loss: 516.5445928012398
Epoch [150/500], Loss: 426.42722746666885
Epoch [160/500], Loss: 406.92131925554327
Epoch [170/500], Loss: 400.53377954954686
Epoch [180/500], Loss: 397.2025483736417
Epoch [190/500], Loss: 398.7136129780437
Epoch [200/500], Loss: 396.6615634245759
Epoch [210/500], Loss: 396.8745437439265
Epoch [220/500], Loss: 397.182392982441
Epoch [230/500], Loss: 396.8359422396649
Epoch [240/500], Loss: 396.88022774250925
Epoch [250/500],

In [8]:
with torch.no_grad():
    y_eval = model.forward(x_test)
    loss = criterion(y_eval,y_test)

In [9]:
num_correct = 0
resid = []
with torch.no_grad():
    for i, data in enumerate(x_test):
        y_val = model.forward(data)
        resid.append(round(y_test[i].item(),2) - round(y_val.item(),2))
        print(f"Predicted: {round(y_val.item(),2)} Expected: {round(y_test[i].item(),2)} Difference: {round(y_test[i].item(),2) - round(y_val.item(),2)}")
        if round(y_val.item(),2) == round(y_test[i].item(),2):
            num_correct += 1
print(f"Correct: {num_correct}")
print(np.std(resid))


Predicted: 4546.92 Expected: 4536.34 Difference: -10.579999999999927
Predicted: 5238.92 Expected: 5234.18 Difference: -4.739999999999782
Predicted: 4755.28 Expected: 4754.63 Difference: -0.6499999999996362
Predicted: 5309.85 Expected: 5307.01 Difference: -2.8400000000001455
Predicted: 5045.39 Expected: 5070.55 Difference: 25.159999999999854
Predicted: 5074.47 Expected: 5071.63 Difference: -2.8400000000001455
Predicted: 5181.1 Expected: 5204.34 Difference: 23.23999999999978
Predicted: 5151.36 Expected: 5178.51 Difference: 27.150000000000546
Predicted: 4885.43 Expected: 4845.65 Difference: -39.780000000000655
Predicted: 5250.77 Expected: 5243.77 Difference: -7.0
Predicted: 4714.96 Expected: 4719.19 Difference: 4.229999999999563
Predicted: 4575.21 Expected: 4585.59 Difference: 10.38000000000011
Predicted: 4892.46 Expected: 4890.97 Difference: -1.4899999999997817
Predicted: 4411.15 Expected: 4411.59 Difference: 0.4400000000005093
Predicted: 4623.98 Expected: 4643.7 Difference: 19.720000000

In [11]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(x_train):
        y_val = model.forward(data)
        print(f"Predicted: {round(y_val.item(),2)} Expected: {round(y_train[i].item(),2)} Difference: {round(y_train[i].item(),2) - round(y_val.item(),2)}")
        if round(y_val.item(),2) == round(y_train[i].item(),2):
            correct += 1

Predicted: 4763.35 Expected: 4780.94 Difference: 17.589999999999236
Predicted: 4343.5 Expected: 4314.6 Difference: -28.899999999999636
Predicted: 4473.2 Expected: 4472.16 Difference: -1.0399999999999636
Predicted: 4561.18 Expected: 4569.78 Difference: 8.599999999999454
Predicted: 5310.54 Expected: 5308.13 Difference: -2.4099999999998545
Predicted: 4441.34 Expected: 4451.14 Difference: 9.800000000000182
Predicted: 4774.96 Expected: 4769.83 Difference: -5.130000000000109
Predicted: 4752.45 Expected: 4768.37 Difference: 15.920000000000073
Predicted: 4518.86 Expected: 4507.66 Difference: -11.199999999999818
Predicted: 5085.67 Expected: 5035.69 Difference: -49.98000000000047
Predicted: 4511.66 Expected: 4478.03 Difference: -33.63000000000011
Predicted: 4383.93 Expected: 4399.77 Difference: 15.840000000000146
Predicted: 4555.42 Expected: 4556.62 Difference: 1.199999999999818
Predicted: 5121.6 Expected: 5127.79 Difference: 6.1899999999996
Predicted: 4356.12 Expected: 4358.24 Difference: 2.119